# お試し実行

In [ ]:
!python --version

Python 3.11.12


In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2025.3.19
Uninstalling unsloth-2025.3.19:
  Successfully uninstalled unsloth-2025.3.19
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-e0re64y2/unsloth_ca9c5d0b181247d3b273ff8d02288778
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-e0re64y2/unsloth_ca9c5d0b181247d3b273ff8d02288778
  Resolved https://github.com/unslothai/unsloth.git to commit f6f5e6397acfb5737dcb2062a72e860cc3657f6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 2.9 MB/s eta 0:00:00
  Created wheel for unsloth: filename=unsloth-2025.3.19-py3-none-any.whl size=192490 sha256=e8c39bc0bfd35a6a0d7f40a06892876956686d4abbbc1af2c674055e347289c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-t4wmmtlf/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221ca

In [ ]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)
print(model.print_trainable_parameters())

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039
None


In [ ]:
data_prompt = """
# 個人情報マスキングタスク

あなたは個人情報のマスキングを行うAIアシスタントです。与えられた日本語テキスト内の個人情報のみを正確にマスキングしてください。

## 重要な制約条件
1. before_maskを出力するときは必ず1回だけであること。何回もbefore_maskを生成してはいけません。
2. after_maskを出力するときは必ず1回だけであること。何回もbefore_maskを生成してはいけません。
3. 元のテキストの構造と文脈を完全に保持すること
4. 元のテキストに存在しない情報を追加しないこと
5. 個人情報以外の部分は一切変更しないこと
6. マスキングは以下の形式で一貫して行うこと:
   - 人名: <マスキング済みの氏名>
   - 会社名: <マスキング済みの会社名>
   - 住所: <マスキング済みの住所>
   - メールアドレス: <マスキング済みのemailアドレス>
   - 電話番号: <マスキング済みの電話番号>
   - 生年月日: <マスキング済みの生年月日>
   - 郵便番号: <マスキング済みの郵便番号>
7. 出力するときは、before_maskとafter_maskだけを1度だけ出力すること。「# 個人情報マスキングタスク」、「## 重要な制約条件」、「## マスキング対象となる個人情報」は出力しないでください。

## マスキング対象となる個人情報
- 氏名（例: 山田太郎、鈴木花子）
- 会社名（例: 株式会社〇〇、〇〇企業）
- 住所（例: 東京都渋谷区〇〇、〇〇県〇〇市）
- メールアドレス（例: xxx@example.com）
- 電話番号（例: 03-xxxx-xxxx、090-xxxx-xxxx）
- 生年月日（例: 1985年5月3日）
- 郵便番号（例: 〒123-4567）

### before_mask:
{}

### after_mask:
{}
"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs       = examples["before_mask"]
    outputs      = examples["after_mask"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
import pandas as pd
all_data_combined_dataset = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/all_data_combined_dataset.csv')

In [ ]:
all_data_combined_dataset

,before_mask,after_mask,source,category,sub_category
0,佐藤花子さんは、2020年5月15日に生まれた息子を連れて、よく千葉県船橋市のショッピングモ...,<マスキング済みの氏名>は、<マスキング済みの生年月日>に生まれた息子を連れて、よく<マスキ...,address,NaN,NaN
1,先日、山本花子さんが勤務する株式会社ニコニコのオフィスへ訪問しました。彼女の電話番号は090...,先日、<マスキング済みの氏名>が勤務する<マスキング済みの会社名>のオフィスへ訪問しました。...,address,NaN,NaN
2,昨日、私の友人である佐藤花子さんと一緒に、新宿のカフェでランチを楽しみました。このカフェの所...,<マスキング済みの氏名>さんと一緒に、新宿のカフェでランチを楽しみました。このカフェの所在地...,address,NaN,NaN
3,田中花子さんは、2010年4月10日に生まれました。彼女は東京都港区南青山にある青山学院大学...,<マスキング済みの氏名>は、<マスキング済みの生年月日>に生まれました。彼女は<マスキング済...,address,NaN,NaN
4,鈴木一郎さんの家族は神奈川県横浜市青葉区美しが丘1丁目2-3に住んでいて、彼は株式会社テクノ...,<マスキング済みの氏名>の家族は<マスキング済みの住所>に住んでいて、彼は<マスキング済みの...,address,NaN,NaN
...,...,...,...,...,...
2866,佐藤健一さんは、2022年4月に株式会社テクノロジーに入社し、東京都新宿区西新宿2丁目1番1...,<マスキング済みの氏名>さんは、<マスキング済みの生年月日>に<マスキング済みの会社名>に入...,post_code,NaN,NaN
2867,先日、株式会社リサーチグループの佐藤花子さんにインタビューを行いました。彼女は東京都渋谷区神...,先日、<マスキング済みの会社名>の<マスキング済みの氏名>さんにインタビューを行いました。彼...,post_code,NaN,NaN
2868,私の友人の鈴木一郎さんは、川崎市幸区にあるソフトウェア開発会社で技術者として働いています。彼...,私の友人の<マスキング済みの氏名>さんは、<マスキング済みの住所>にある<マスキング済みの会...,post_code,NaN,NaN
2869,株式会社日本製鉄の技術者である佐藤花子さんは、東京都港区芝公園6丁目9番2号に住んでいます。...,<マスキング済みの会社名>の技術者である<マスキング済みの氏名>さんは、<マスキング済みの住...,post_code,NaN,NaN


In [ ]:
training_data = Dataset.from_pandas(all_data_combined_dataset)
training_data = training_data.map(formatting_prompt, batched=True)

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

In [ ]:
training_data

Dataset({
    features: ['before_mask', 'after_mask', 'source', 'category', 'sub_category', 'text'],
    num_rows: 2871
})

In [ ]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=training_data,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=32,
        num_train_epochs=10,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
        # wandbを無効化するための設定を追加
        report_to=["tensorboard"],  # または report_to="none"
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2871 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,871 | Num Epochs = 10 | Total steps = 220
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 32
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 32 x 1) = 128
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.335300
2,1.341200
3,1.323300
4,1.240400
5,1.104300
6,0.958200
7,0.856800
8,0.721000
9,0.647200
10,0.573600


TrainOutput(global_step=220, training_loss=0.349038850041953, metrics={'train_runtime': 2103.0837, 'train_samples_per_second': 13.651, 'train_steps_per_second': 0.105, 'total_flos': 2.0663052818413978e+17, 'train_loss': 0.349038850041953})

In [ ]:
print("35分で完了")

35分で完了


In [ ]:
# モデルの保存
model_path = "personal_info_masking_model_202504161437"
trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"モデルを {model_path} に保存しました")

モデルを personal_info_masking_model_202504161437 に保存しました


In [ ]:
text = "先日、私の友人である佐藤健一が誕生日を迎えました。彼は1985年3月14日生まれで、毎年その日には特別なパーティーを開きます。今年は渋谷区道玄坂のレストランで集まり、約20人ほどの友人が参加しました。パーティーの準備をしているとき、佐藤さんの会社の同僚である斉藤花子が手伝いに来てくれました。彼女は新宿にあるABC株式会社で働いています。パーティーでは、美味しい料理や楽しいゲームがあって、皆が笑顔で溢れていました。そして、待ちに待ったケーキの登場です。ケーキには「お誕生日おめでとう、健一！」と書かれており、彼は感激していました。パーティーが終わる頃には、記念に皆で写真を撮っています。その時に、佐藤さんの携帯電話の番号090-1234-5678が画面に表示されていました。パーティーのスナップショットを友人たちと共有するために、佐藤は自分のメールアドレスを使いましたが、後でみんなで食事に行くことを約束しました。,<マスキング済みの氏名>が誕生日を迎えました。彼は<マスキング済みの生年月日>生まれで、毎年その日には特別なパーティーを開きます。今年は<マスキング済みの住所>のレストランで集まり、約20人ほどの友人が参加しました。パーティーの準備をしているとき、<マスキング済みの氏名>が手伝いに来てくれました。彼女は<マスキング済みの会社名>で働いています。パーティーでは、美味しい料理や楽しいゲームがあって、皆が笑顔で溢れていました。そして、待ちに待ったケーキの登場です。ケーキには「お誕生日おめでとう、<マスキング済みの氏名>！」と書かれており、彼は感激していました。パーティーが終わる頃には、記念に皆で写真を撮っています。その時に、<マスキング済みの電話番号>が画面に表示されていました。パーティーのスナップショットを友人たちと共有するために、<マスキング済みのemailアドレス>を使いましたが、後でみんなで食事に行くことを約束しました。"

model = FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 5020, use_cache = True)
answer=tokenizer.batch_decode(outputs)
answer = answer[0].split("### Response:")[-1]
print("Answer of the question is:", answer)

Answer of the question is: <|begin_of_text|>
# 個人情報マスキングタスク

あなたは個人情報のマスキングを行うAIアシスタントです。与えられた日本語テキスト内の個人情報のみを正確にマスキングしてください。

## 重要な制約条件
1. before_maskを出力するときは必ず1回だけであること。何回もbefore_maskを生成してはいけません。
2. after_maskを出力するときは必ず1回だけであること。何回もbefore_maskを生成してはいけません。
3. 元のテキストの構造と文脈を完全に保持すること
4. 元のテキストに存在しない情報を追加しないこと
5. 個人情報以外の部分は一切変更しないこと
6. マスキングは以下の形式で一貫して行うこと:
   - 人名: <マスキング済みの氏名>
   - 会社名: <マスキング済みの会社名>
   - 住所: <マスキング済みの住所>
   - メールアドレス: <マスキング済みのemailアドレス>
   - 電話番号: <マスキング済みの電話番号>
   - 生年月日: <マスキング済みの生年月日>
   - 郵便番号: <マスキング済みの郵便番号>
7. 出力するときは、before_maskとafter_maskだけを1度だけ出力すること。「# 個人情報マスキングタスク」、「## 重要な制約条件」、「## マスキング対象となる個人情報」は出力しないでください。

## マスキング対象となる個人情報
- 氏名（例: 山田太郎、鈴木花子）
- 会社名（例: 株式会社〇〇、〇〇企業）
- 住所（例: 東京都渋谷区〇〇、〇〇県〇〇市）
- メールアドレス（例: xxx@example.com）
- 電話番号（例: 03-xxxx-xxxx、090-xxxx-xxxx）
- 生年月日（例: 1985年5月3日）
- 郵便番号（例: 〒123-4567）

### before_mask:
先日、私の友人である佐藤健一が誕生日を迎えました。彼は1985年3月14日生まれで、毎年その日には特別なパーティーを開きます。今年は渋谷区道玄坂のレストランで集まり、約20人ほどの友人が参加しました。パーティーの準備をしているとき、佐藤さんの会社の同僚である斉藤花子が手伝いに来

# Llama-3.2-1B-bnb-4bit

In [ ]:
!pip install evaluate rouge-score

In [ ]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported
import random
import evaluate

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:

# ハイパーパラメータ
max_seq_length = 2048
learning_rate = 2e-4
lr_scheduler_type = "cosine"
weight_decay = 0.05
r_lora = 32
lora_alpha = 32
lora_dropout = 0.05
random_seed = 42

# シード固定
import numpy as np
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# モデルロード
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

# LoRA設定
model = FastLanguageModel.get_peft_model(
    model,
    r=r_lora,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state=random_seed,
    loftq_config=None,
)
print(model.print_trainable_parameters())

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 16 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 22,544,384 || all params: 1,258,358,784 || trainable%: 1.7916
None


In [ ]:
# 改善されたプロンプトテンプレート
data_prompt = """
# 個人情報マスキングタスク

あなたは個人情報のマスキングを行うAIアシスタントです。日本語テキスト内の個人情報のみを正確にマスキングしてください。

## マスキング対象の個人情報と形式
- 人名: <マスキング済みの氏名>
- 会社名: <マスキング済みの会社名>
- 住所: <マスキング済みの住所>
- メールアドレス: <マスキング済みのemailアドレス>
- 電話番号: <マスキング済みの電話番号>
- 生年月日: <マスキング済みの生年月日>
- 郵便番号: <マスキング済みの郵便番号>

## 制約条件
- 個人情報以外の部分は一切変更しない
- 元のテキストの構造と文脈を完全に保持する
- マスキング後の出力はbefore_maskのテキストの構造と文脈を完全に保持して個人情報のみをマスキングしafter_maskに出力すること

### before_mask:
{}

### after_mask:
{}
"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs = examples["before_mask"]
    outputs = examples["after_mask"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# データセットをロード
training_data = Dataset.from_pandas(all_data_combined_dataset)

# データセットのクレンジングと検証関数
def validate_dataset(example):
    """データセットの品質チェック"""
    is_valid = True
    reason = ""

    # 入力が空でないか確認
    if not example["before_mask"] or len(example["before_mask"]) < 10:
        is_valid = False
        reason = "入力テキストが短すぎるか空です"

    # 出力が入力よりも極端に短くないか確認
    elif len(example["after_mask"]) < len(example["before_mask"]) * 0.5:
        is_valid = False
        reason = "出力が入力に比べて極端に短いです"

    # 個人情報のマスキングキーワードが含まれているか確認
    elif "<マスキング済み" not in example["after_mask"] and len(example["before_mask"]) > 100:
        is_valid = False
        reason = "マスキングが行われていない可能性があります"

    return {"is_valid": is_valid, "reason": reason}

# データセットの検証
validation_results = training_data.map(validate_dataset)
valid_data = Dataset.from_dict({
    k: [v for i, v in enumerate(validation_results[k]) if validation_results["is_valid"][i]]
    for k in training_data.column_names
})

print(f"元のデータセットサイズ: {len(training_data)}")
print(f"クレンジング後のデータセットサイズ: {len(valid_data)}")
print(f"除外された例: {len(training_data) - len(valid_data)}")

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

元のデータセットサイズ: 2871
クレンジング後のデータセットサイズ: 1283
除外された例: 1588


In [ ]:
# クレンジング後のデータセットを使用
training_data = valid_data

# データオーグメンテーション
def data_augmentation(example):
    """データの多様性を増やすための簡単なオーグメンテーション"""
    text = example["before_mask"]
    # ランダムに段落の順序を入れ替える（50%の確率で）
    if random.random() > 0.5 and "\n\n" in text:
        paragraphs = text.split("\n\n")
        if len(paragraphs) > 1:
            random.shuffle(paragraphs)
            text = "\n\n".join(paragraphs)

    return {"before_mask": text, "after_mask": example["after_mask"]}

# データの20%をオーグメント
augmentation_split = training_data.train_test_split(test_size=0.2, seed=random_seed)
augmented_data = augmentation_split["test"].map(data_augmentation)
final_data = Dataset.from_dict({
    "before_mask": training_data["before_mask"] + augmented_data["before_mask"],
    "after_mask": training_data["after_mask"] + augmented_data["after_mask"]
})

print(f"オーグメンテーション後のデータセットサイズ: {len(final_data)}")

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

オーグメンテーション後のデータセットサイズ: 1540


In [ ]:
# プロンプトフォーマット適用
training_data = final_data.map(formatting_prompt, batched=True)

# トレーニングデータと評価データを分割
train_eval_split = training_data.train_test_split(test_size=0.05, seed=random_seed)
train_dataset = train_eval_split["train"]
eval_dataset = train_eval_split["test"]

# 評価指標の定義
def compute_metrics(eval_preds):
    rouge = evaluate.load("rouge")
    predictions, labels = eval_preds

    # トークンIDをテキストに変換
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGEスコアの計算
    result = rouge.compute(predictions=predictions, references=labels, use_stemmer=True)

    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
    }

# warmup_stepsの計算
warmup_steps = int(0.1 * (len(train_dataset) // (4 * 32)))

Map:   0%|          | 0/1540 [00:00<?, ? examples/s]

In [ ]:
# トレーニング設定
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,
    packing=True,
    # compute_metricsはここでは使用できません
    args=TrainingArguments(
        learning_rate=learning_rate,
        lr_scheduler_type=lr_scheduler_type,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=32,
        num_train_epochs=5,
        # 評価関連パラメータを削除
        # evaluation_strategy="steps",
        # eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=3,
        # load_best_model_at_end=True,
        # metric_for_best_model="eval_loss",
        # greater_is_better=False,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        logging_first_step=True,
        optim="adamw_8bit",
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        output_dir="output",
        seed=random_seed,
        report_to=["tensorboard"],
    ),
)

# トレーニング実行
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/1463 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/77 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,463 | Num Epochs = 5 | Total steps = 55
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 32
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 32 x 1) = 128
 "-____-"     Trainable parameters = 22,544,384/1,000,000,000 (2.25% trained)


Step,Training Loss
1,1.387800
10,0.748100
20,0.488300
30,0.445600
40,0.415800
50,0.398000


TrainOutput(global_step=55, training_loss=0.5013744354248046, metrics={'train_runtime': 494.0413, 'train_samples_per_second': 14.806, 'train_steps_per_second': 0.111, 'total_flos': 3.417244432301261e+16, 'train_loss': 0.5013744354248046})

In [ ]:
# モデルの保存
model_path = "/content/drive/MyDrive/ColabNotebooks/personal_info_masking_model"
trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"モデルを {model_path} に保存しました")

モデルを /content/drive/MyDrive/ColabNotebooks/personal_info_masking_model に保存しました


In [ ]:
from transformers import AutoTokenizer
from unsloth import FastLanguageModel

# 保存したモデルのパス
# model_path = "/content/drive/MyDrive/ColabNotebooks/personal_info_masking_model"

# # モデルとトークナイザーの読み込み
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name=model_path,
#     max_seq_length=2048,
#     load_in_4bit=True,
#     dtype=None,
# )

# # 推論用に設定
# model = model.eval()

# 以下は推論の例
def generate_masked_text(input_text):
    # プロンプトテンプレート
    prompt = f"""
# 個人情報マスキングタスク

あなたは個人情報のマスキングを行うAIアシスタントです。日本語テキスト内の個人情報のみを正確にマスキングしてください。

## マスキング対象の個人情報と形式
- 人名: <マスキング済みの氏名>
- 会社名: <マスキング済みの会社名>
- 住所: <マスキング済みの住所>
- メールアドレス: <マスキング済みのemailアドレス>
- 電話番号: <マスキング済みの電話番号>
- 生年月日: <マスキング済みの生年月日>
- 郵便番号: <マスキング済みの郵便番号>

## 制約条件
- 個人情報以外の部分は一切変更しない
- 元のテキストの構造と文脈を完全に保持する
- マスキング後の出力はbefore_maskのテキストの構造と文脈を完全に保持して個人情報のみをマスキングしafter_maskに出力すること

### before_mask:
{input_text}

### after_mask:
"""

    # トークナイズしてモデルに入力
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 生成
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=1024,
            temperature=0.1,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True,
        )

    # 生成されたテキストをデコード
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # "after_mask:" より後の部分を抽出
    try:
        masked_text = generated_text.split("### after_mask:")[1].strip()
    except IndexError:
        # 適切な形式で出力されなかった場合
        masked_text = "出力形式エラー: " + generated_text

    return masked_text

ModuleNotFoundError: No module named 'unsloth'

In [ ]:
result

'先日、私の友人である<マスキング済みの氏名>が誕生日を迎えました。彼は<マスキング済みの生年月日>生まれで、毎年その日には特別なパーティーを開きます。今年は<マスキング済みの住所>のレストランで集まり、約20人ほどの友人が参加しました。パーティーの準備をしているとき、<マスキング済みの氏名>が手伝いに来てくれました。彼女は<マスキング済みの会社名>で働いています。パーティーでは、美味しい料理や楽しいゲームがあって、皆が笑顔で溢れていました。そして、待ちに待ったケーキの登場です。ケーキには「お誕生日おめでとう、<マスキング済みの氏名>！」と書かれており、彼は感激していました。パーティーが終わる頃には、記念に皆で写真を撮っています。その時に、<マスキング済みの電話番号>が画面に表示されていました。パーティーのスナップショットを友人たちと共有するために、<マスキング済みのemailアドレス>を使いましたが、後でみんなで食事に行くことを約束しました。'

In [ ]:
test_text = """
山田太郎様
住所: 東京都渋谷区神南1-2-3 アップルマンション501
電話: 03-1234-5678
メール: yamada.taro@example.com
株式会社テックソリューション
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

山田太郎様
住所: 東京都渋谷区神南1-2-3 アップルマンション501
電話: 03-1234-5678
メール: yamada.taro@example.com
株式会社テックソリューション


マスキング後:
<マスキング済みの氏名>様
住所: <マスキング済みの住所>
電話: <マスキング済みの電話番号>
メール: <マスキング済みのemailアドレス>
<マスキング済みの会社名>


# unsloth/Llama-3.2-3B-Instruct-bnb-4bit

In [ ]:
# モデルロード
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

# LoRA設定
model = FastLanguageModel.get_peft_model(
    model,
    r=r_lora,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state=random_seed,
    loftq_config=None,
)
print(model.print_trainable_parameters())

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 48,627,712 || all params: 3,261,377,536 || trainable%: 1.4910
None


In [ ]:
# 改善されたプロンプトテンプレート
data_prompt = """
# 個人情報マスキングタスク

あなたは個人情報のマスキングを行うAIアシスタントです。日本語テキスト内の個人情報のみを正確にマスキングしてください。

## マスキング対象の個人情報と形式
- 人名: <マスキング済みの氏名>
- 会社名: <マスキング済みの会社名>
- 住所: <マスキング済みの住所>
- メールアドレス: <マスキング済みのemailアドレス>
- 電話番号: <マスキング済みの電話番号>
- 生年月日: <マスキング済みの生年月日>
- 郵便番号: <マスキング済みの郵便番号>

## 制約条件
- 個人情報以外の部分は一切変更しない
- 元のテキストの構造と文脈を完全に保持する
- マスキング後の出力はbefore_maskのテキストの構造と文脈を完全に保持して個人情報のみをマスキングしafter_maskに出力すること

### before_mask:
{}

### after_mask:
{}
"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs = examples["before_mask"]
    outputs = examples["after_mask"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# データセットをロード
training_data = Dataset.from_pandas(all_data_combined_dataset)

# データセットのクレンジングと検証関数
def validate_dataset(example):
    """データセットの品質チェック"""
    is_valid = True
    reason = ""

    # 入力が空でないか確認
    if not example["before_mask"] or len(example["before_mask"]) < 10:
        is_valid = False
        reason = "入力テキストが短すぎるか空です"

    # 出力が入力よりも極端に短くないか確認
    elif len(example["after_mask"]) < len(example["before_mask"]) * 0.5:
        is_valid = False
        reason = "出力が入力に比べて極端に短いです"

    # 個人情報のマスキングキーワードが含まれているか確認
    elif "<マスキング済み" not in example["after_mask"] and len(example["before_mask"]) > 100:
        is_valid = False
        reason = "マスキングが行われていない可能性があります"

    return {"is_valid": is_valid, "reason": reason}

# データセットの検証
validation_results = training_data.map(validate_dataset)
valid_data = Dataset.from_dict({
    k: [v for i, v in enumerate(validation_results[k]) if validation_results["is_valid"][i]]
    for k in training_data.column_names
})

print(f"元のデータセットサイズ: {len(training_data)}")
print(f"クレンジング後のデータセットサイズ: {len(valid_data)}")
print(f"除外された例: {len(training_data) - len(valid_data)}")

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

元のデータセットサイズ: 2871
クレンジング後のデータセットサイズ: 1283
除外された例: 1588


In [ ]:
# クレンジング後のデータセットを使用
training_data = valid_data

# データオーグメンテーション
def data_augmentation(example):
    """データの多様性を増やすための簡単なオーグメンテーション"""
    text = example["before_mask"]
    # ランダムに段落の順序を入れ替える（50%の確率で）
    if random.random() > 0.5 and "\n\n" in text:
        paragraphs = text.split("\n\n")
        if len(paragraphs) > 1:
            random.shuffle(paragraphs)
            text = "\n\n".join(paragraphs)

    return {"before_mask": text, "after_mask": example["after_mask"]}

# データの20%をオーグメント
augmentation_split = training_data.train_test_split(test_size=0.2, seed=random_seed)
augmented_data = augmentation_split["test"].map(data_augmentation)
final_data = Dataset.from_dict({
    "before_mask": training_data["before_mask"] + augmented_data["before_mask"],
    "after_mask": training_data["after_mask"] + augmented_data["after_mask"]
})

print(f"オーグメンテーション後のデータセットサイズ: {len(final_data)}")

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

オーグメンテーション後のデータセットサイズ: 1540


In [ ]:
# プロンプトフォーマット適用
training_data = final_data.map(formatting_prompt, batched=True)

# トレーニングデータと評価データを分割
train_eval_split = training_data.train_test_split(test_size=0.05, seed=random_seed)
train_dataset = train_eval_split["train"]
eval_dataset = train_eval_split["test"]

# 評価指標の定義
def compute_metrics(eval_preds):
    rouge = evaluate.load("rouge")
    predictions, labels = eval_preds

    # トークンIDをテキストに変換
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGEスコアの計算
    result = rouge.compute(predictions=predictions, references=labels, use_stemmer=True)

    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
    }

# warmup_stepsの計算
warmup_steps = int(0.1 * (len(train_dataset) // (4 * 32)))

Map:   0%|          | 0/1540 [00:00<?, ? examples/s]

In [ ]:
# トレーニング設定
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,
    packing=True,
    # compute_metricsはここでは使用できません
    args=TrainingArguments(
        learning_rate=learning_rate,
        lr_scheduler_type=lr_scheduler_type,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=32,
        num_train_epochs=5,
        # 評価関連パラメータを削除
        # evaluation_strategy="steps",
        # eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=3,
        # load_best_model_at_end=True,
        # metric_for_best_model="eval_loss",
        # greater_is_better=False,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        logging_first_step=True,
        optim="adamw_8bit",
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        output_dir="output",
        seed=random_seed,
        report_to=["tensorboard"],
    ),
)

# トレーニング実行
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/1463 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/77 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,463 | Num Epochs = 5 | Total steps = 55
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 32
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 32 x 1) = 128
 "-____-"     Trainable parameters = 48,627,712/3,000,000,000 (1.62% trained)


Step,Training Loss
1,1.323400
10,0.705800
20,0.472800
30,0.434000
40,0.406900
50,0.387600


TrainOutput(global_step=55, training_loss=0.4838840614665638, metrics={'train_runtime': 1069.1325, 'train_samples_per_second': 6.842, 'train_steps_per_second': 0.051, 'total_flos': 1.01776045628928e+17, 'train_loss': 0.4838840614665638})

In [ ]:
# モデルの保存
model_path = "/content/drive/MyDrive/ColabNotebooks/Llama_3_2_3B_Instruct_bnb_4bit_personal_info_masking_model"
trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"モデルを {model_path} に保存しました")

モデルを /content/drive/MyDrive/ColabNotebooks/Llama_3_2_3B_Instruct_bnb_4bit_personal_info_masking_model に保存しました


In [ ]:
!pip install -U bitsandbytes

!pip install -U transformers accelerate

!pip install -U bitsandbytes transformers accelerate peft trl

  Using cached trl-0.16.1-py3-none-any.whl.metadata (12 kB)
Using cached trl-0.16.1-py3-none-any.whl (336 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.3.17 requires trl!=0.15.0,!=0.9.0,!=0.9.1,!=0.9.2,!=0.9.3,<=0.15.2,>=0.7.9, but you have trl 0.16.1 which is incompatible.


In [ ]:
from transformers import AutoTokenizer
from unsloth import FastLanguageModel

# 保存したモデルのパス
model_path = "/content/drive/MyDrive/ColabNotebooks/Llama_3_2_3B_Instruct_bnb_4bit_personal_info_masking_model"

# モデルとトークナイザーの読み込み
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=None,
)

# 推論用に設定
model = model.eval()

# 以下は推論の例
def generate_masked_text(input_text):
    # プロンプトテンプレート
    prompt = f"""
# 個人情報マスキングタスク

あなたは個人情報のマスキングを行うAIアシスタントです。日本語テキスト内の個人情報のみを正確にマスキングしてください。

## マスキング対象の個人情報と形式
- 人名: <マスキング済みの氏名>
- 会社名: <マスキング済みの会社名>
- 住所: <マスキング済みの住所>
- メールアドレス: <マスキング済みのemailアドレス>
- 電話番号: <マスキング済みの電話番号>
- 生年月日: <マスキング済みの生年月日>
- 郵便番号: <マスキング済みの郵便番号>

## 制約条件
- before_maskの個人情報以外の部分は一切変更しないでください。
- before_maskのテキストの構造と文脈を完全に保持して、個人情報のみをマスキングし、after_maskに出力すること
- マスキング後の出力はbefore_maskのテキストの構造と文脈を完全に保持して個人情報のみをマスキングしafter_maskに出力すること

### before_mask:
{input_text}

### after_mask:
"""

    # トークナイズしてモデルに入力
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 生成
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=1024,
            temperature=0.1,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True,
        )

    # 生成されたテキストをデコード
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # "after_mask:" より後の部分を抽出
    try:
        masked_text = generated_text.split("### after_mask:")[1].strip()
    except IndexError:
        # 適切な形式で出力されなかった場合
        masked_text = "出力形式エラー: " + generated_text

    return masked_text

<ipython-input-4-15ea2c906d57>:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
test_text = """
山田太郎様
住所: 東京都渋谷区神南1-2-3 アップルマンション501
電話: 03-1234-5678
メール: yamada.taro@example.com
株式会社テックソリューション
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

山田太郎様
住所: 東京都渋谷区神南1-2-3 アップルマンション501
電話: 03-1234-5678
メール: yamada.taro@example.com
株式会社テックソリューション


マスキング後:
<マスキング済みの氏名>様
住所: <マスキング済みの住所> アップルマンション501
電話: <マスキング済みの電話番号>
メール: <マスキング済みのemailアドレス>
<マスキング済みの会社名>


In [ ]:
test_text = """
佐藤健一さんは株式会社aozoraに勤務しており、東京都渋谷区桜丘町26-1にあるオフィスで働いています。氏名や住所は個人情報として扱われるため、本人の許可を得ずに開示することはありません。また、連絡先として使用されるメールアドレスはsato@example.comで、電話番号は080-1234-5678です。最近、彼は1985年5月10日に生まれたことを思い出し、誕生日を祝う計画を立てています。
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

佐藤健一さんは株式会社aozoraに勤務しており、東京都渋谷区桜丘町26-1にあるオフィスで働いています。氏名や住所は個人情報として扱われるため、本人の許可を得ずに開示することはありません。また、連絡先として使用されるメールアドレスはsato@example.comで、電話番号は080-1234-5678です。最近、彼は1985年5月10日に生まれたことを思い出し、誕生日を祝う計画を立てています。


マスキング後:
<マスキング済みの氏名>さんは<マスキング済みの会社名>に勤務しており、<マスキング済みの住所>にあるオフィスで働いています。氏名や住所は個人情報として扱われるため、本人の許可を得ずに開示することはありません。また、連絡先として使用されるメールアドレスは<マスキング済みのemailアドレス>で、電話番号は<マスキング済みの電話番号>です。最近、彼は<マスキング済みの生年月日>に生まれたことを思い出し、誕生日を祝う計画を立てています。


In [ ]:
test_text = """
こんにちは。私は、東京都新宿区に住む佐藤健一と申します。生年月日は平成5年8月15日で、現在はXYZ株式会社でマーケティング担当として働いています。私の仕事では、クライアントとの打ち合わせが多く、特に最近では新しいプロジェクトのためにナショナルクライアントとの連携を強化しています。このプロジェクトは非常に重要で、私たちの会社にとっても大きなチャンスです。また、私の連絡先は090-9876-5432、メールアドレスはsato.kenichi@example.comです。よろしくお願いいたします。

先日、クライアントとミーティングを行った際には、彼らのニーズを把握し、具体的な提案を行うことができました。特に、自社の製品の特長やユニークなセールスポイントを効果的に伝えることが重要だと思っています。そして、プロジェクトチーム全体で連携を取りながら、戦略を立て、実行に移していく必要があります。私たちのチームは、クリエイティブでありながらもフレキシブルに対応できるスタッフで構成されており、それぞれの意見を尊重し合いながら進めています。

また、私の趣味はサイクリングです。週末には友人と一緒に多摩川沿いを自転車で走ったり、時には遠出をして自然を楽しんだりします。健康のためにも運動は大切だと思っており、普段からできるだけ体を動かすようにしています。このようなリフレッシュがあるためこそ、仕事に集中できるのだと感じています。

それに加えて、最近はテクノロジーやデジタルマーケティングの勉強にも力を入れています。業界が進化している中で、最新のトレンドを把握することは自分の成長にもつながりますので、オンラインの講座を受けることもあります。これからもスキルアップを図り、より高いパフォーマンスを発揮できるよう努力し続けます。
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

こんにちは。私は、東京都新宿区に住む佐藤健一と申します。生年月日は平成5年8月15日で、現在はXYZ株式会社でマーケティング担当として働いています。私の仕事では、クライアントとの打ち合わせが多く、特に最近では新しいプロジェクトのためにナショナルクライアントとの連携を強化しています。このプロジェクトは非常に重要で、私たちの会社にとっても大きなチャンスです。また、私の連絡先は090-9876-5432、メールアドレスはsato.kenichi@example.comです。よろしくお願いいたします。

先日、クライアントとミーティングを行った際には、彼らのニーズを把握し、具体的な提案を行うことができました。特に、自社の製品の特長やユニークなセールスポイントを効果的に伝えることが重要だと思っています。そして、プロジェクトチーム全体で連携を取りながら、戦略を立て、実行に移していく必要があります。私たちのチームは、クリエイティブでありながらもフレキシブルに対応できるスタッフで構成されており、それぞれの意見を尊重し合いながら進めています。

また、私の趣味はサイクリングです。週末には友人と一緒に多摩川沿いを自転車で走ったり、時には遠出をして自然を楽しんだりします。健康のためにも運動は大切だと思っており、普段からできるだけ体を動かすようにしています。このようなリフレッシュがあるためこそ、仕事に集中できるのだと感じています。

それに加えて、最近はテクノロジーやデジタルマーケティングの勉強にも力を入れています。業界が進化している中で、最新のトレンドを把握することは自分の成長にもつながりますので、オンラインの講座を受けることもあります。これからもスキルアップを図り、より高いパフォーマンスを発揮できるよう努力し続けます。


マスキング後:
こんにちは。私は、<マスキング済みの住所>に住む<マスキング済みの氏名>と申します。生年月日は<マスキング済みの生年月日>で、現在は<マスキング済みの会社名>でマーケティング担当として働いています。私の仕事では、クライアントとの打ち合わせが多く、特に最近では新しいプロジェクトのためにナショナルクライアントとの連携を強化しています。このプロジェクトは非常に重要で、私たちの会社にとても大きなチャンスです。また、私の連絡先は<マスキング済

In [ ]:
test_text = """
こんにちは、私は佐藤花子と申します。生まれたのは1990年3月15日で、今は東京都中央区銀座4丁目に住んでいます。趣味は読書や料理をすることです。現在は株式会社ABCのマーケティング部に所属しており、様々なプロジェクトに取り組んでいます。もし何か連絡が必要な場合は、私のメールアドレスにご連絡ください。メールアドレスは hanako.sato@example.com です。また、電話番号は090-9876-5432です。私の夢は、自分の会社を持って、好きなことを仕事にすることです。そのためにも、日々自分自身を磨く努力をしています。また、最近はフィットネスにも通い始め、健康維持にも力を入れるようにしています。季節の変わり目には体調を崩しやすいということを聞いたので、十分に気を付けています。この間は友人と旅行に行く計画を立てていて、福岡に行こうと思っています。美味しいものをもたくさん食べたいですね。旅先では新しい体験をするのが一番の楽しみです。最後に何か質問があれば、ぜひ連絡をください。
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

こんにちは、私は佐藤花子と申します。生まれたのは1990年3月15日で、今は東京都中央区銀座4丁目に住んでいます。趣味は読書や料理をすることです。現在は株式会社ABCのマーケティング部に所属しており、様々なプロジェクトに取り組んでいます。もし何か連絡が必要な場合は、私のメールアドレスにご連絡ください。メールアドレスは hanako.sato@example.com です。また、電話番号は090-9876-5432です。私の夢は、自分の会社を持って、好きなことを仕事にすることです。そのためにも、日々自分自身を磨く努力をしています。また、最近はフィットネスにも通い始め、健康維持にも力を入れるようにしています。季節の変わり目には体調を崩しやすいということを聞いたので、十分に気を付けています。この間は友人と旅行に行く計画を立てていて、福岡に行こうと思っています。美味しいものをもたくさん食べたいですね。旅先では新しい体験をするのが一番の楽しみです。最後に何か質問があれば、ぜひ連絡をください。


マスキング後:
こんにちは、私は<マスキング済みの氏名>と申します。生まれたのは<マスキング済みの生年月日>で、今は<マスキング済みの住所>に住んでいます。趣味は読書や料理をすることです。現在は<マスキング済みの会社名>のマーケティング部に所属しており、様々なプロジェクトに取り組んでいます。もし何か連絡が必要な場合は、私のメールアドレスにご連絡ください。メールアドレスは <マスキング済みのemailアドレス> でした。また、電話番号は<マスキング済みの電話番号>です。私の夢は、自分の会社を持って、好きなことを仕事にすることです。そのためにも、日々自分SELFを磨く努力をしています。また、最近はフィットネスにも通い始め、健康維持にも力を入れるようにしています。季節の変わり目には体調を崩しやすいということを聞いたので、十分に気を付けるようにしています。この間は友人と旅行に行く計画を立てていて、福岡に行こうと思っています。美味しいものをもたくさん食べたいですね。旅先では新しい体験をするのが一番の楽しみです。最後に何か質問があれば、ぜひ連絡をください。


In [ ]:
# 保存したモデルのパス
model_path = "/content/drive/MyDrive/ColabNotebooks/Llama_3_2_3B_Instruct_bnb_4bit_personal_info_masking_model"

# モデルとトークナイザーの読み込み
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=4096,
    load_in_4bit=True,
    dtype=None,
)

# 推論用に設定
model = model.eval()

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
test_text = """
こんにちは。今日は私の友人、新宿区に住む鈴木花子の誕生日を祝うために、特別なサプライズパーティーを企画しています。彼女は1985年6月15日に生まれ、現在の年齢は38歳です。彼女の好きなケーキ屋は、渋谷区にある「スイートシェフ」で、毎年彼女の誕生日には必ずケーキを頼みます。今年もおいしいフルーツタルトを予約しました。パーティーには私の他に、彼女の大学時代の友人である佐藤一郎と、会社の同僚である高橋健太も参加予定です。特に佐藤は彼女とは長い付き合いで、毎年彼女の誕生日にはプレゼントを贈ってお祝いしています。今回は何を贈るかまだ決めていないようですが、皆で話し合って決めるつもりです。

パーティーは彼女が仕事を終えた後、午後7時から私の家で開催します。私の住所は東京都品川区中延3-5-7ですが、サプライズなので彼女には知られないようにしています。参加者全員が集まった後は、ちょっとしたゲームを楽しむ予定です。その後、ケーキを切り分けて、彼女の素敵な一年を祝います。最後に、彼女への感謝の気持ちを伝るために、みんなでメッセージカードを書こうと思っています。

私の連絡先は080-9876-5432で、万が一のことを考えて、参加者には私のメールアドレスも伝えておきました。私のメールアドレスはmyer_email_address@example.comです。鈴木さんへのサプライズが成功するように、しっかりと準備を進めたいと思います。それでは、これから買い物に出かける準備をします。
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

こんにちは。今日は私の友人、新宿区に住む鈴木花子の誕生日を祝うために、特別なサプライズパーティーを企画しています。彼女は1985年6月15日に生まれ、現在の年齢は38歳です。彼女の好きなケーキ屋は、渋谷区にある「スイートシェフ」で、毎年彼女の誕生日には必ずケーキを頼みます。今年もおいしいフルーツタルトを予約しました。パーティーには私の他に、彼女の大学時代の友人である佐藤一郎と、会社の同僚である高橋健太も参加予定です。特に佐藤は彼女とは長い付き合いで、毎年彼女の誕生日にはプレゼントを贈ってお祝いしています。今回は何を贈るかまだ決めていないようですが、皆で話し合って決めるつもりです。

パーティーは彼女が仕事を終えた後、午後7時から私の家で開催します。私の住所は東京都品川区中延3-5-7ですが、サプライズなので彼女には知られないようにしています。参加者全員が集まった後は、ちょっとしたゲームを楽しむ予定です。その後、ケーキを切り分けて、彼女の素敵な一年を祝います。最後に、彼女への感謝の気持ちを伝るために、みんなでメッセージカードを書こうと思っています。

私の連絡先は080-9876-5432で、万が一のことを考えて、参加者には私のメールアドレスも伝えておきました。私のメールアドレスはmyer_email_address@example.comです。鈴木さんへのサプライズが成功するように、しっかりと準備を進めたいと思います。それでは、これから買い物に出かける準備をします。


マスキング後:
こんにちは。今日は私の友人、新宿区に住む<マスキング済みの氏名>の誕生日を祝うために、特別なサプライズパーティーを企画しています。彼女は<マスキング済みの生年月日>に生まれ、現在の年齢は38歳です。彼女の好きなケーキ屋は、渋谷区にある<マスキング済みの会社名>で、毎年彼女の誕生日には必ずケーキを頼みます。今年もおいしいフルーツタルトを予約しました。パーティーには私の他に、彼女の大学時代の友人である<マスキング済みの氏名>と、会社の同僚である<マスキング済みの氏名>も参加予定です。特に<マスキング済みの氏名>は彼女とは長い付き合いで、毎年彼女の誕生日にはプレゼントを贈ってお祝いしています。今回は何を贈るかまだ決めていないようですが、皆で話し合って決めるつもりです。



In [ ]:
test_text = """
こんにちは。私の名前は佐藤一郎で、株式会社 SoftBank Japan でエンジニアとして働いています。サーバーやネットワークに関するプロジェクトを多く担当しており、最近は特にデータベースの最適化やセキュリティについて深く学んでいます。仕事の関係で、東京の新宿区に住んでいますが、愛知県の出身です。生まれた年月日は1985年の5月3日です。休日はハイキングや読書を楽しんでおり、特に歴史小説が好きです。また、私のメールアドレスは ichiro.sato@example.com で、緊急の場合は、070-9876-5432 までお電話いただければすぐに対応可能です。この業界では常に新しい技術が生まれるため、自己学習を続けることが重要だと思っています。近々、海外のカンファレンスにも参加する予定で、そこでのネットワーキングを通じて、さらなるスキルアップを目指しています。日本の技術者として誇りを持って仕事ができることを嬉しく思っています。
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

こんにちは。私の名前は佐藤一郎で、株式会社 SoftBank Japan でエンジニアとして働いています。サーバーやネットワークに関するプロジェクトを多く担当しており、最近は特にデータベースの最適化やセキュリティについて深く学んでいます。仕事の関係で、東京の新宿区に住んでいますが、愛知県の出身です。生まれた年月日は1985年の5月3日です。休日はハイキングや読書を楽しんでおり、特に歴史小説が好きです。また、私のメールアドレスは ichiro.sato@example.com で、緊急の場合は、070-9876-5432 までお電話いただければすぐに対応可能です。この業界では常に新しい技術が生まれるため、自己学習を続けることが重要だと思っています。近々、海外のカンファレンスにも参加する予定で、そこでのネットワーキングを通じて、さらなるスキルアップを目指しています。日本の技術者として誇りを持って仕事ができることを嬉しく思っています。


マスキング後:
こんにちは。私の名前は<マスキング済みの氏名>で、<マスキング済みの会社名> でエンジニアとして働いています。サーバーやネットワークに関するプロジェクトを多く担当しており、最近は特にデータベースの最適化やセキュリティについて深く学んでいます。仕事の関係で、<マスキング済みの住所>に住んでいますが、愛知県の出身です。生まれた年月日は<マスキング済みの生年月日>です。休日はハイキングや読書を楽しんでおり、特に歴史小説が好きです。また、私のメールアドレスは <マスキング済みのemailアドレス> で、緊急の場合は、<マスキング済みの電話番号> にお電話いただければすぐに対応可能です。この業界では常に新しい技術が生まれるため、自己学習を続けることが重要だと思っています。近々、海外のカンファレンスにも参加する予定で、そこでのネットワーキングを通じて、さらなるスキルアップを目指しています。日本の技術者として誇りを持って仕事ができることを嬉しく思っています。


In [ ]:
test_text = """
こんにちは。私の名前は佐藤一郎で、株式会社 SoftBank Japan でエンジニアとして働いています。2025年4月17日までに契約書を提出して下さい。よろしくお願い致します。
"""

result = generate_masked_text(test_text)
print("元のテキスト:")
print(test_text)
print("\nマスキング後:")
print(result)

元のテキスト:

こんにちは。私の名前は佐藤一郎で、株式会社 SoftBank Japan でエンジニアとして働いています。2025年4月17日までに契約書を提出して下さい。よろしくお願い致します。


マスキング後:
<マスキング済みの氏名>で、<マスキング済みの会社名>でエンジニアとして働いています。<マスキング済みの生年月日>までに契約書を提出してください。よろくお願いします。
